### Step 1. Download the college dataset:
#### gsutil cp gs://cs327e-open-access/fs_college.zip .
#### unzip fs_college.zip

### Step 2: Run code samples in sequence to populate Firestore database

In [1]:
import pandas as pd
from google.cloud import firestore
db = firestore.Client()

In [2]:
batch = db.batch()

df = pd.read_csv('fs_college/class.csv', sep=',', header=0, lineterminator='\n')
rows = df.values.tolist()

for row in rows:
    
    record = {}
    record['cno'] = row[0]
    record['cname'] = row[1]
    record['credits'] = row[2]
    
    class_ref = db.collection('class').document(row[0])
    
    batch.set(class_ref, record)
    
batch.commit()

[update_time {
   seconds: 1634339171
   nanos: 326045000
 },
 update_time {
   seconds: 1634339171
   nanos: 326045000
 },
 update_time {
   seconds: 1634339171
   nanos: 326045000
 },
 update_time {
   seconds: 1634339171
   nanos: 326045000
 },
 update_time {
   seconds: 1634339171
   nanos: 326045000
 },
 update_time {
   seconds: 1634339171
   nanos: 326045000
 },
 update_time {
   seconds: 1634339171
   nanos: 326045000
 },
 update_time {
   seconds: 1634339171
   nanos: 326045000
 },
 update_time {
   seconds: 1634339171
   nanos: 326045000
 }]

In [3]:
batch = db.batch()

df = pd.read_csv('fs_college/student.csv', sep=',', header=0, lineterminator='\n')
student_rows = df.values.tolist()

df = pd.read_csv('fs_college/takes.csv', sep=',', header=0, lineterminator='\n')
takes_rows = df.values.tolist()

for student_row in student_rows:
    
    student_record = {}
    student_record['sid'] = student_row[0]
    student_record['fname'] = student_row[1]
    student_record['lname'] = student_row[2]
    student_record['dob'] = student_row[3]
    student_record['status'] = student_row[4]
    
    student_ref = db.collection('student').document(student_row[0])
    
    batch.set(student_ref, student_record)
    
    for takes_row in takes_rows:
        if student_row[0] == takes_row[0]:
            
            class_record = {}
            class_record['cno'] = takes_row[1]
            class_record['cname'] = takes_row[2]
            class_record['credits'] = takes_row[3]
            class_record['grade'] = takes_row[4]
            
            classes_ref = student_ref.collection('classes').document(takes_row[1])
        
            batch.set(classes_ref, class_record)
    
    
    batch.commit()
    

In [4]:
batch = db.batch()

df = pd.read_csv('fs_college/instructor.csv', sep=',', header=0, lineterminator='\n')
instructor_rows = df.values.tolist()

df = pd.read_csv('fs_college/teaches.csv', sep=',', header=0, lineterminator='\n')
teaches_rows = df.values.tolist()

for instructor_row in instructor_rows:
    
    instructor_record = {}
    instructor_record['tid'] = instructor_row[0]
    instructor_record['instructor_name'] = instructor_row[1]
    instructor_record['dept'] = instructor_row[2]
    
    instructor_ref = db.collection('instructor').document(instructor_row[0])
    
    batch.set(instructor_ref, instructor_record)
    
    for teaches_row in teaches_rows:
        
        if instructor_row[0] == teaches_row[0]:
            
            class_record = {}
            class_record['cno'] = teaches_row[1]
            class_record['cname'] = teaches_row[2]
            class_record['credits'] = teaches_row[3]
            
            classes_ref = instructor_ref.collection('classes').document(teaches_row[1])
        
            batch.set(classes_ref, class_record)
    
    batch.commit()

### Step 3: Run some queries

In [5]:
student_ref = db.collection('student').document('paulg')
result = student_ref.get()

if result.exists:
    print(f'{result.id} => {result.to_dict()}')
else:
    print('No such student')

paulg => {'dob': '2000-09-17', 'lname': 'Gore', 'fname': 'Paul', 'status': 'CUR', 'sid': 'paulg'}


In [6]:
student_ref = db.collection('student')
query = student_ref.where('status', '==', 'CUR')
results = query.stream()

for result in results:
    print(f'{result.id} => {result.to_dict()}')

aprilz => {'fname': 'April', 'sid': 'aprilz', 'dob': '2000-10-01', 'lname': 'Lopez', 'status': 'CUR'}
bzen26 => {'sid': 'bzen26', 'lname': 'Zen', 'dob': '1998-04-22', 'fname': 'Biswa', 'status': 'CUR'}
jc => {'fname': 'James', 'sid': 'jc', 'dob': '2000-04-22', 'status': 'CUR', 'lname': 'Cowe'}
jerryh => {'status': 'CUR', 'fname': 'Jerry', 'sid': 'jerryh', 'lname': 'Hargrove', 'dob': '1999-01-03'}
kev18 => {'fname': 'Kevin', 'status': 'CUR', 'sid': 'kev18', 'dob': '1999-10-05', 'lname': 'Lin'}
paulg => {'status': 'CUR', 'lname': 'Gore', 'fname': 'Paul', 'sid': 'paulg', 'dob': '2000-09-17'}
sudeepa4 => {'status': 'CUR', 'fname': 'Sudeepa', 'sid': 'sudeepa4', 'lname': 'Roy', 'dob': '2001-10-01'}


In [7]:
query = db.collection('instructor').document('mitra').collection('classes')

results = query.stream()

for result in results:
    print(f'{result.id} => {result.to_dict()}')

CS313E => {'credits': 3, 'cno': 'CS313E', 'cname': 'Elements of Software Engineering'}
CS329E => {'cno': 'CS329E', 'credits': 3, 'cname': 'Elements of Web Programming'}


In [11]:
classes = db.collection_group('classes').where('credits', '==', 3)
docs = classes.stream()
for doc in docs:
    print(f'{doc.id} => {doc.to_dict()}')

CS303E => {'cno': 'CS303E', 'credits': 3, 'cname': 'Elements of Computers and Programming'}
CS326E => {'cno': 'CS326E', 'credits': 3, 'cname': 'Elements of Networking'}
M328K => {'credits': 3, 'cno': 'M328K', 'cname': 'Intro to Number Theory'}
CS313E => {'credits': 3, 'cname': 'Elements of Software Engineering', 'cno': 'CS313E'}
CS329E => {'cno': 'CS329E', 'cname': 'Elements of Web Programming', 'credits': 3}
M362K => {'cno': 'M362K', 'cname': 'Probability I', 'credits': 3}
M362K => {'cname': 'Probability I', 'credits': 3, 'cno': 'M362K'}
CS327E => {'credits': 3, 'cno': 'CS327E', 'cname': 'Elements of Databases'}
M358K => {'cname': 'Applied Statistics', 'cno': 'M358K', 'credits': 3}


In [9]:
instructor_ref = db.collection('instructor')
results = instructor_ref.stream()

for result in results:
    print(f'{result.id} => {result.to_dict()}')
    sresults = instructor_ref.document(result.id).collection('classes').stream()
    
    for sresult in sresults:
        print(f'{sresult.id} => {sresult.to_dict()}')

bulko => {'instructor_name': 'Bill Bulko', 'dept': 'Computer Science', 'tid': 'bulko'}
CS303E => {'cno': 'CS303E', 'cname': 'Elements of Computers and Programming', 'credits': 3}
cannata => {'instructor_name': 'Phil Cannata', 'tid': 'cannata', 'dept': 'Computer Science'}
CS326E => {'cno': 'CS326E', 'cname': 'Elements of Networking', 'credits': 3}
downing => {'tid': 'downing', 'dept': 'Computer Science', 'instructor_name': 'Glenn Downing'}
fares => {'dept': 'Computer Science', 'instructor_name': 'Fares Fraij', 'tid': 'fares'}
koch => {'tid': 'koch', 'dept': 'Mathematics', 'instructor_name': 'Hans Koch'}
M328K => {'cno': 'M328K', 'cname': 'Intro to Number Theory', 'credits': 3}
mitra => {'dept': 'Computer Science', 'instructor_name': 'Shyamal Mitra', 'tid': 'mitra'}
CS313E => {'cno': 'CS313E', 'cname': 'Elements of Software Engineering', 'credits': 3}
CS329E => {'credits': 3, 'cname': 'Elements of Web Programming', 'cno': 'CS329E'}
mueller => {'dept': 'Mathematics', 'tid': 'mueller', 'in